In [1]:
import pandas as pd
import numpy as np
import os
import networkx as nx
import matplotlib.pyplot as plt
import plotly
import plotly.plotly as py
import plotly.offline as pyoff
from plotly.graph_objs import *
from plotly import tools
import time
import math

In [2]:
pyoff.init_notebook_mode(connected=True)

# EEG Channels Animation

In [3]:
alcoholic_data = pd.read_feather('data/alcoholics')
control_data = pd.read_feather('data/controls')
with open('data/channel_map.txt', 'r') as f:
    lines = [line.strip().split()[2:] for line in f.readlines()]
    channel_map_dict = {line[0].upper(): [float(elem) for elem in line[1:]] for line in lines}

In [4]:
grouped_alcoholic_data = alcoholic_data.groupby(['channel', 'timepoint'])['signal'].mean().reset_index()
grouped_control_data = control_data.groupby(['channel', 'timepoint'])['signal'].mean().reset_index()
grouped_alcoholic_data.shape, grouped_control_data.shape

((15616, 3), (15616, 3))

In [5]:
def animate_raw_eeg(alcoholic_data, control_data, channel_map_dict):
    '''
    Visualizes 60 chanels of raw  eeg signals averaged across subjects within each group
    (alcoholics/controls) over the course of a 1 second (256 sample trial)
    '''
    
    def get_vals(df, tp, typ):
        now = df[df['timepoint'] == tp]
        coords = np.array([channel_map_dict[chan] for chan in now['channel']])
        colors = [sig for sig in now['signal']]
        return coords, colors, now['channel']
    
    def get_frame(tp):
        coords_alc, colors_alc, chans_alc = get_vals(alcoholic_data, tp, 'alc')
        trace_alc = Scatter(
            name='Alcoholics',
            x = coords_alc[:,0],
            y = coords_alc[:,1],
            hovertext = chans_alc,
            mode='markers',
            marker=dict(
                size='25',
                color = colors_alc, #set color equal to a variable
                cmin = cmin,
                cmax = cmax,
                colorscale = 'BlueRed',
                colorbar = dict(
                    thickness=20,
                    title='Electrical Signal',
                    xanchor='left',
                    titleside='right'
                ),
                showscale=True
            )
        )
        
        coords_ctrl, colors_ctrl, chans_ctrl = get_vals(control_data, tp, 'ctrl')
        trace_ctrl =  Scatter(
            name='Controls',
            x = coords_ctrl[:,0],
            y = coords_ctrl[:,1],
            hovertext = chans_ctrl,
            mode='markers',
            marker=dict(
                size='20',
                color = colors_ctrl, #set color equal to a variable
                cmin = cmin,
                cmax = cmax,
                colorscale = 'BlueRed',
                showscale=False
            )
        )
        
        layout = Layout(
            showlegend=False,
            xaxis=dict(
            domain=[0, 0.45]
            ),
            xaxis2=dict(
                domain=[0.55, 9]
            ),
            yaxis2=dict(
                anchor='x2'
            )
        )
        
        

        data = [trace_alc, trace_ctrl]
        return data, layout


    
    tp_max = 256
    all_tps = range(tp_max)
    cmin = np.min(np.vstack([alcoholic_data['signal'], control_data['signal']]))
    cmax = np.max(np.vstack([alcoholic_data['signal'], control_data['signal']]))
    
    fig = tools.make_subplots(rows=1, cols=2, subplot_titles=['Alcoholics', 'Controls'])
    
    data = [dict(data=get_frame(tp)[0], layout=get_frame(tp)[1]) for tp in all_tps]
    fig.append_trace(data[0]['data'][0], 1, 1)
    fig.append_trace(data[0]['data'][1], 1, 2)

    sliders_dict = {
        'active': 0,
        'yanchor': 'top',
        'xanchor': 'left',
        'currentvalue': {
            'font': {'size': 20},
            'prefix': 'Time Bin:',
            'visible': True,
            'xanchor': 'right'
        },
        'transition': {'duration': 300, 'easing': 'cubic-in-out'},
        'pad': {'b': 10, 't': 50},
        'len': 0.9,
        'x': 0.1,
        'y': 0,
        'steps': []
    }
    fig['frames'] = list()
    for tp in all_tps:
        frame = {'data': [], 'name': str(tp)}
        frame['data'].append(data[tp]['data'][0])
        frame['data'].append(data[tp]['data'][1])
        fig['frames'].append(frame)
        slider_step = {'args': [
                [tp],
                {'frame': {'duration': 50, 'redraw': False},
                 'mode': 'immediate',
               'transition': {'duration': 50}}
             ],
             'label': tp,
             'method': 'animate'}
        sliders_dict['steps'].append(slider_step)
    fig['layout']['sliders'] = [sliders_dict]
    fig['layout']['updatemenus'] = [
            {
                'buttons': [
                    {
                        'args': [None, {'fromcurrent': True, 'transition': {'duration': 50, 'easing': 'quadratic-in-out'}}],
                        'label': 'Play',
                        'method': 'animate'
                    },
                    {
                        'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                        'transition': {'duration': 0}}],
                        'label': 'Pause',
                        'method': 'animate'
                    }
                ],
                'direction': 'left',
                'pad': {'r': 10, 't': 87},
                'showactive': False,
                'type': 'buttons',
                'x': 0.1,
                'xanchor': 'right',
                'y': 0,
                'yanchor': 'top'
            }
        ]
    fig['layout']['showlegend'] = False
    fig['layout']['title']='Electrical Signals'
    fig['layout']['titlefont']=dict(size=28)
    pyoff.plot(fig, filename='direct_comparison')

In [6]:
animate_raw_eeg(grouped_alcoholic_data, grouped_control_data, channel_map_dict)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



/home/jmcarpenter/anaconda2/envs/fastai/lib/python3.6/site-packages/plotly/offline/offline.py:459: UserWarning:

Your filename `direct_comparison` didn't end with .html. Adding .html to the end of your file.



# Subject Network

In [7]:
alcoholic_nodes = [2,  3,  5,  6,  8,  9, 12, 15, 16, 18]
control_nodes = [0, 1,  4,  7, 10, 11, 13, 14, 17, 19]

def load_files(tp_max):
        Gs = list()
        for tp in range(tp_max):
            FILEPATH = os.path.join(PATH, 'subj_adjmat_tp{}.csv'.format(tp))
            ggmod = np.loadtxt(open(FILEPATH, "rb"), delimiter=",", skiprows=1)
            Gs.append(nx.from_numpy_matrix(ggmod))
        return Gs

In [8]:

PATH = 'data/'

def animate_subject_network(animation_name='subject_net'):
    '''
    Visualizes the gaussian graphical model representation of the subjects over the course
    of 1 second (60 overlapping 16 sample time bins)
    '''

    alcoholic_nodes = [2, 3, 5, 6, 8, 9, 12, 15, 16, 18]
    control_nodes = [0, 1, 4, 7, 10, 11, 13, 14, 17, 19]
        
    pos = dict()
    subs = np.arange(20)
    x = np.cos(2*math.pi*subs/20)
    y = np.sin(2*math.pi*subs/20)
    # Controls
    for a, p in zip(alcoholic_nodes, np.arange(10)):
        pos[a] = np.array([x[p], y[p]])
        
    for c, p in zip(control_nodes, np.arange(10, 20)):
        pos[c] = np.array([x[p], y[p]])
    
    def get_frame(tp):
        G = Gs[tp]
        
        for n, p in pos.items():
            G.node[n]['pos'] = p
        
        edge_trace = Scatter(
            x=[],
            y=[],
            line=Line(width=1,color='#888'),
            hoverinfo='none',
            mode='lines')

        for edge in G.edges():
            x0, y0 = G.node[edge[0]]['pos']
            x1, y1 = G.node[edge[1]]['pos']
            edge_trace['x'] += [x0, x1, None]
            edge_trace['y'] += [y0, y1, None]

        node_trace_alc = Scatter(
            x=[],
            y=[],
            text=[],
            mode='markers',
            hoverinfo='text',
            marker=Marker(
                showscale=True,
                # colorscale options
                # 'Greys' | 'Greens' | 'Bluered' | 'Hot' | 'Picnic' | 'Portland' |
                # Jet' | 'RdBu' | 'Blackbody' | 'Earth' | 'Electric' | 'YIOrRd' | 'YIGnBu'
                colorscale='RdBu',
                reversescale=False,
                color=[],
                line = dict(
                    color = 'red',
                    width = 10
                  ),
                size=50,
                cmin=-3,
                cmax=3,
                colorbar=dict(
                    thickness=25,
                    title='Alcoholic-Control Connections',
                    xanchor='left',
                    titleside='right'
                )
            )
        )
                
        node_trace_ctrl = Scatter(
            x=[],
            y=[],
            text=[],
            mode='markers',
            hoverinfo='text',
            marker=Marker(
                showscale=False,
                # colorscale options
                # 'Greys' | 'Greens' | 'Bluered' | 'Hot' | 'Picnic' | 'Portland' |
                # Jet' | 'RdBu' | 'Blackbody' | 'Earth' | 'Electric' | 'YIOrRd' | 'YIGnBu'
                colorscale='RdBu',
                reversescale=True,
                color=[],
                line = dict(
                    color = 'blue',
                    width = 10
                  ),
                size=50,
                cmin=-3,
                cmax=3,
                colorbar=dict(
                    thickness=25,
                    title='Alcoholic-Control Connections',
                    xanchor='left',
                    titleside='right'
                )
            )
        )
            
        for node, adjacencies in enumerate(G.adjacency_list()):
            x, y = G.node[node]['pos']
            if node in alcoholic_nodes:
                node_trace_alc['x'].append(x)
                node_trace_alc['y'].append(y)
                within_adjacencies = [adj for adj in adjacencies if adj in alcoholic_nodes]
                between_adjacencies = [adj for adj in adjacencies if adj in control_nodes]
                node_trace_alc['text'].append('Alcohlic ~ Subject %d\nAlcoholic connections: %d\nControl connections: %d' 
                                               % (node, len(within_adjacencies), len(between_adjacencies)))
                
                node_trace_alc['marker']['color'].append(len(within_adjacencies) - len(between_adjacencies))
            elif node in control_nodes:
                node_trace_ctrl['x'].append(x)
                node_trace_ctrl['y'].append(y)
                within_adjacencies = [adj for adj in adjacencies if adj in control_nodes]
                between_adjacencies = [adj for adj in adjacencies  if adj in alcoholic_nodes]
                node_trace_ctrl['text'].append('Control ~ Subject %d\nAlcoholic connections: %d \nControl connections: %d' 
                                               % (node, len(between_adjacencies), len(within_adjacencies)))
                 
                node_trace_ctrl['marker']['color'].append(len(within_adjacencies) - len(between_adjacencies))

        data=Data([node_trace_alc, node_trace_ctrl, edge_trace])
        
        layout=Layout(
            title='Subject Network',
            titlefont=dict(size=28),
            showlegend=False,
            hovermode='closest',
            margin=dict(b=20,l=5,r=5,t=40),
            height=650,
            width=650,
            xaxis=XAxis(showgrid=False, zeroline=False, showticklabels=False),
            yaxis=YAxis(showgrid=False, zeroline=False, showticklabels=False),
            updatemenus = [
            {
                'buttons': [
                    {
                        'args': [None, {'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                        'label': 'Play',
                        'method': 'animate'
                    },
                    {
                        'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                        'transition': {'duration': 0}}],
                        'label': 'Pause',
                        'method': 'animate'
                    }
                ],
                'direction': 'left',
                'pad': {'r': 10, 't': 87},
                'showactive': False,
                'type': 'buttons',
                'x': 0.1,
                'xanchor': 'right',
                'y': 0,
                'yanchor': 'top'
            }
        ])
        
        return {'data': data, 'layout': layout}

    tp_max = 61 
    all_tps = range(tp_max)
    Gs = load_files(tp_max)
    fig = get_frame(0)
    fig['frames'] = list()
    data = [get_frame(tp) for tp in all_tps]
    sliders_dict = {
        'active': 0,
        'yanchor': 'top',
        'xanchor': 'left',
        'currentvalue': {
            'font': {'size': 20},
            'prefix': 'Time Bin:',
            'visible': True,
            'xanchor': 'right'
        },
        'transition': {'duration': 300, 'easing': 'cubic-in-out'},
        'pad': {'b': 10, 't': 50},
        'len': 0.9,
        'x': 0.1,
        'y': 0,
        'steps': []
    }
    for tp in all_tps:
        frame = {'data': [], 'name': str(tp)}
        frame['data'].append(data[tp]['data'][0])
        frame['data'].append(data[tp]['data'][1])
        frame['data'].append(data[tp]['data'][2])
        frame['layout'] = data[tp]['layout']
        fig['frames'].append(frame)
        slider_step = {'args': [
                [tp],
                {'frame': {'duration': 300, 'redraw': False},
                 'mode': 'immediate',
               'transition': {'duration': 300}}
             ],
             'label': tp,
             'method': 'animate'}
        sliders_dict['steps'].append(slider_step)
    fig['layout']['sliders'] = [sliders_dict]
    pyoff.plot(fig, filename='subject_network')

In [9]:
animate_subject_network()

/home/jmcarpenter/anaconda2/envs/fastai/lib/python3.6/site-packages/plotly/offline/offline.py:459: UserWarning:

Your filename `subject_network` didn't end with .html. Adding .html to the end of your file.



# Channel Network

In [10]:
alcoholic_nodes = [2,  3,  5,  6,  8,  9, 12, 15, 16, 18]
control_nodes = [0, 1,  4,  7, 10, 11, 13, 14, 17, 19]

def load_chan_files(tp_max, alc_or_ctrl):
        Gs = list()
        for tp in range(tp_max):
            FILEPATH = os.path.join(PATH, '{}chan_adjmat_tp{}.csv'.format(alc_or_ctrl, tp))
            ggmod = np.loadtxt(open(FILEPATH, "rb"), delimiter=",", skiprows=1)
            Gs.append(nx.from_numpy_matrix(ggmod))
        return Gs

In [11]:

PATH = 'data/'

def animate_channel_network(animation_name='subject_net'):
    '''
    Visualizes the gaussian graphical model representation of the 60 channels over 
    the course of 1 second (60 overlapping 16 sample time bins)
    '''
    channel_order = np.load('data/channel_order.npy')
    xs, ys = np.zeros(61), np.zeros(61)
    pos = dict()
    for n, chan in enumerate(channel_order):
        xs[n], ys[n], _ = channel_map_dict[chan]
        pos[n] = np.array([xs[n], ys[n]])
        
    def get_frame(tp):
        G_alc = Gs_alc[tp]
        G_ctrl = Gs_ctrl[tp]
        
        for n, p in pos.items():
            G_alc.node[n]['pos'] = p
            G_ctrl.node[n]['pos'] = p
        
        edge_trace_alc = Scatter(
            x=[],
            y=[],
            line=Line(width=1,color='#888'),
            hoverinfo='none',
            mode='lines')

        for edge in G_alc.edges():
            x0, y0 = G_alc.node[edge[0]]['pos']
            x1, y1 = G_alc.node[edge[1]]['pos']
            edge_trace_alc['x'] += [x0, x1, None]
            edge_trace_alc['y'] += [y0, y1, None]
            
        edge_trace_ctrl = Scatter(
            x=[],
            y=[],
            line=Line(width=1,color='#888'),
            hoverinfo='none',
            mode='lines')

        for edge in G_ctrl.edges():
            x0, y0 = G_ctrl.node[edge[0]]['pos']
            x1, y1 = G_ctrl.node[edge[1]]['pos']
            edge_trace_ctrl['x'] += [x0, x1, None]
            edge_trace_ctrl['y'] += [y0, y1, None]

        node_trace_alc = Scatter(
            x=[],
            y=[],
            text=[],
            mode='markers',
            hoverinfo='text',
            marker=Marker(
                showscale=True,
                # colorscale options
                # 'Greys' | 'Greens' | 'Bluered' | 'Hot' | 'Picnic' | 'Portland' |
                # Jet' | 'RdBu' | 'Blackbody' | 'Earth' | 'Electric' | 'YIOrRd' | 'YIGnBu'
                colorscale='Hot',
                reversescale=True,
                color=[],
                size=20,
                cmin=0,
                cmax=cmax,
                colorbar=dict(
                    thickness=25,
                    title='Channel Connections',
                    xanchor='left',
                    titleside='right'
                ),
                line=dict(width=2)))
        
        node_trace_ctrl = Scatter(
            x=[],
            y=[],
            text=[],
            mode='markers',
            hoverinfo='text',
            marker=Marker(
                showscale=False,
                # colorscale options
                # 'Greys' | 'Greens' | 'Bluered' | 'Hot' | 'Picnic' | 'Portland' |
                # Jet' | 'RdBu' | 'Blackbody' | 'Earth' | 'Electric' | 'YIOrRd' | 'YIGnBu'
                colorscale='Hot',
                reversescale=True,
                color=[],
                size=20,
                cmin=0,
                cmax=cmax,
                colorbar=dict(
                    thickness=25,
                    title='Channel Connections',
                    xanchor='left',
                    titleside='right'
                ),
                line=dict(width=2)))

        for node, adjacencies in enumerate(G_alc.adjacency_list()):
            x, y = G_alc.node[node]['pos']
            node_trace_alc['x'].append(x)
            node_trace_alc['y'].append(y)
            node_trace_alc['text'].append('Channel %s\nConnections %d' 
                                           % (channel_order[node], len(adjacencies)))
            node_trace_alc['marker']['color'].append(len(adjacencies))
                                          
        for node, adjacencies in enumerate(G_ctrl.adjacency_list()):
            x, y = G_ctrl.node[node]['pos']
            node_trace_ctrl['x'].append(x)
            node_trace_ctrl['y'].append(y)
            node_trace_ctrl['text'].append('Channel %s\nConnections %d' 
                                           % (channel_order[node], len(adjacencies)))
            node_trace_ctrl['marker']['color'].append(len(adjacencies))
                
        data = Data([node_trace_alc, edge_trace_alc, node_trace_ctrl, edge_trace_ctrl])
        
        layout = Layout(
            title='Channel Network',
            titlefont=dict(size=28),
            showlegend=False,
            hovermode='closest',
            margin=dict(b=20,l=5,r=5,t=40),

            updatemenus = [
                {
                    'buttons': [
                        {
                            'args': [None, {'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                            'label': 'Play',
                            'method': 'animate'
                        },
                        {
                            'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                            'transition': {'duration': 0}}],
                            'label': 'Pause',
                            'method': 'animate'
                        }
                    ],
                    'direction': 'left',
                    'showactive': False,
                    'pad': {'r':10, 't':87},
                    'type': 'buttons',
                    'x': 0.1,
                    'xanchor': 'right',
                    'y': 0,
                    'yanchor': 'top'
                }
            ])
        
        return {'data': data, 'layout': layout}

    tp_max = 61    
    all_tps = range(tp_max)
    Gs_alc = load_chan_files(tp_max, 'alc')     
    Gs_ctrl = load_chan_files(tp_max, 'ctrl')
    cmax_alc = np.max([np.max([len(adj) for adj in G_alc.adjacency_list()]) for G_alc in Gs_alc])
    cmax_ctrl = np.max([np.max([len(adj) for adj in G_ctrl.adjacency_list()]) for G_ctrl in Gs_ctrl])
    cmax = max(cmax_alc, cmax_ctrl)
    fig = tools.make_subplots(rows=1, cols=2, subplot_titles=['Alcoholics', 'Controls'])
    data = [get_frame(tp) for tp in all_tps]
    fig.append_trace(data[0]['data'][0], 1, 1)
    fig.append_trace(data[0]['data'][1], 1, 1)
    fig.append_trace(data[0]['data'][2], 1, 2)
    fig.append_trace(data[0]['data'][3], 1, 2)
    sliders_dict = {
        'active': 0,
        'yanchor': 'top',
        'xanchor': 'left',
        'currentvalue': {
            'font': {'size': 20},
            'prefix': 'Time Bin:',
            'visible': True,
            'xanchor': 'right'
        },
        'transition': {'duration': 300, 'easing': 'cubic-in-out'},
        'pad': {'b': 10, 't': 50},
        'len': 0.9,
        'x': 0.1,
        'y': 0,
        'steps': []
    }
    for tp in all_tps:
        frame = {'data': [], 'name': str(tp)}
        frame['data'].append(data[tp]['data'][0])
        frame['data'].append(data[tp]['data'][1])
        frame['data'].append(data[tp]['data'][2])
        frame['data'].append(data[tp]['data'][3])
        frame['layout'] = data[tp]['layout']
        fig['frames'].append(frame)
        slider_step = {'args': [
                [tp],
                {'frame': {'duration': 300, 'redraw': False},
                 'mode': 'immediate',
               'transition': {'duration': 300}}
             ],
             'label': tp,
             'method': 'animate'}
        sliders_dict['steps'].append(slider_step)
    fig['layout']['sliders'] = [sliders_dict]
    
    
    pyoff.plot(fig, filename='channel_network')

In [12]:
animate_channel_network()

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



/home/jmcarpenter/anaconda2/envs/fastai/lib/python3.6/site-packages/plotly/offline/offline.py:459: UserWarning:

Your filename `channel_network` didn't end with .html. Adding .html to the end of your file.

